In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy import stats
from scipy.stats import norm
from copy import deepcopy

In [185]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
import lightgbm as lgb

In [54]:
df = pd.read_csv('train.csv')

In [69]:
df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [208]:
df_copy = deepcopy(df)

# Feature engineering

In [202]:
to_drop = ['target', 'ID_code']
extra_features = []
for column in df.drop(['target', 'ID_code'], axis=1):
    to_drop.append(column)
    for col in df.drop(to_drop, axis=1):
        corr = df['target'].corr(df[[column, col]].mean(axis=1))
        if  corr >= 0.08 or corr <= -0.08:
            extra_features.append([column, col])

In [203]:
to_drop = ['target', 'ID_code']
extra_features_norm = []
for column in df.drop(['target', 'ID_code'], axis=1):
    to_drop.append(column)
    for col in df.drop(to_drop, axis=1):
        corr = df['target'].corr(np.sqrt(df[column]**2+df[col]**2))
        if  corr >= 0.08 or corr <= -0.08:
            extra_features_norm.append([column, col])

In [195]:
to_drop = ['target', 'ID_code']
extra_features_scale = []
for column in df.drop(['target', 'ID_code'], axis=1):
    corr = df['target'].corr(np.round(df[column]*100))
    if  corr >= 0.05 or corr <= -0.05:
        extra_features_scale.append(column)

In [8]:
to_drop = ['target', 'ID_code']
extra_features_binning = []
for column in df.drop(['target', 'ID_code'], axis=1):
    corr = df['target'].corr(pd.qcut(
        df[column],
        [0, 0.25, 0.5, 0.75, 1], labels=False))
    if  corr >= 0.05 or corr <= -0.05:
        extra_features_binning.append(column)

In [204]:
for x, y in extra_features:
    df_copy['{}_{}_mean'.format(x,y)] = df[[x, y]].mean(axis=1)

for x, y in extra_features_norm:
    df_copy['{}_{}_norm'.format(x,y)] = np.sqrt(df[x]**2+df[y]**2)

# for x in extra_features_scale:
#     df_copy['{}_scale'.format(x)] = np.round(df[column]*100)

# for x in extra_features_binning:
#     df_copy['{}_binning'.format(x)] = pd.qcut(df[column], [0, 0.25, 0.5, 0.75, 1], labels=False)

In [231]:
df_copy.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


# Splitting

In [269]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2129)
for train_id, valid_id in split.split(df_copy, df_copy['target']):
    df_train = df_copy.loc[train_id]
    df_valid = df_copy.loc[valid_id]

In [270]:
X_train = df_train.drop(['ID_code', 'target'], axis=1)
y_train = df_train['target']

X_valid = df_valid.drop(['ID_code', 'target'], axis=1)
y_valid = df_valid['target']

In [134]:
X_tr = df_train.drop(['ID_code', 'target'], axis=1)
y_tr = df_train['target']

In [272]:
X = df_copy.drop(['ID_code', 'target'], axis=1)
y = df_copy['target']

In [212]:
df_test = pd.read_csv('test.csv')

In [213]:
X_test = df_test.drop(['ID_code'], axis=1)

#  Ensemble of different resampled datasets

Used ensemble of different resampled datasets to deal with unbalanced data(negative class has N times more samples than positive class). All negative samples in train dataset splitted into N chunks of equal length. Then create N datasets in which each dataset contains all positive samples and one of the chunks with negative samples. Build N models on N created datasets. 

## CatBoost

In [264]:
negatives = np.array_split(df_copy[df_copy['target']==0], 9)

predictions = []

models = []
scalers = []

for i, negative in enumerate(negatives):
    print(f'Iteration {i+1}')
    train = pd.concat([df_copy[df_copy['target']==1], negative], axis=0).sample(frac=1)

    X_train = train.drop(['ID_code', 'target'], axis=1)
    y_train = train['target']
        
    catboost = CatBoostClassifier(n_estimators=10000,
                                  max_depth=3,
                                  learning_rate=0.025,
                                  reg_lambda=25,
                                  logging_level='Silent',
                                  random_seed=63)

    catboost.fit(X_train, y_train)
    
    models.append(catboost)

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [132]:
negatives = np.array_split(df_train[df_train['target']==0], 10)

predictions = []

models = []

for i, negative in enumerate(negatives):
    print(f'Iteration {i+1}')
    train = pd.concat([df_train[df_train['target']==1], negative], axis=0).sample(frac=1)

    X_train = train.drop(['ID_code', 'target'], axis=1)
    y_train = train['target']
    catboost = CatBoostClassifier(n_estimators=10000,
                                  max_depth=3,
                                  learning_rate=0.025,
                                  reg_lambda=25,
                                  logging_level='Silent',
                                  random_seed=63)

    catboost.fit(X_train, y_train)
        
    y_pred_train = catboost.predict_proba(X_train)[:,1]
    y_pred_valid = catboost.predict_proba(X_valid)[:,1]
    
    models.append(catboost)
    predictions.append(y_pred_valid)

    print('ROC AUC train: ', roc_auc_score(y_train, y_pred_train))
    print('ROC AUC valid: ', roc_auc_score(y_valid, y_pred_valid))

Iteration 1
ROC AUC train:  0.9503300872223455
ROC AUC valid:  0.8955294331566181
Iteration 2
ROC AUC train:  0.9528647544464397
ROC AUC valid:  0.8953415800375555
Iteration 3
ROC AUC train:  0.950256435570037
ROC AUC valid:  0.8949398090149586
Iteration 4
ROC AUC train:  0.9486606721498952
ROC AUC valid:  0.8946182718978759
Iteration 5
ROC AUC train:  0.9501715329749129
ROC AUC valid:  0.8938265730823372
Iteration 6
ROC AUC train:  0.951310751553658
ROC AUC valid:  0.8949874792242236
Iteration 7
ROC AUC train:  0.9514719912881622
ROC AUC valid:  0.8951656116305632
Iteration 8
ROC AUC train:  0.9501351838082842
ROC AUC valid:  0.8949261543864888
Iteration 9
ROC AUC train:  0.9505784655903085
ROC AUC valid:  0.8942136385886024
Iteration 10
ROC AUC train:  0.9510293793576216
ROC AUC valid:  0.8956837477426652


In [122]:
print('ROC AUC valid: ', roc_auc_score(y_valid, np.mean(predictions, axis=0)))

ROC AUC valid:  0.8982718633071441


In [125]:
# 5000, learning_rate=0.05
print('ROC AUC valid: ', roc_auc_score(y_valid, np.mean(predictions, axis=0)))

ROC AUC valid:  0.8987975215639423


In [133]:
# 10000, learning_rate=0.025
print('ROC AUC valid: ', roc_auc_score(y_valid, np.mean(predictions, axis=0)))

ROC AUC valid:  0.8990693350921878


In [253]:
# 10000, learning_rate=0.025
print('ROC AUC valid: ', roc_auc_score(y_valid, np.mean(
    [
        catboost_valid_pred[0],
        catboost_valid_pred[1],
        catboost_valid_pred[2],
        catboost_valid_pred[3],

        catboost_valid_pred[5],
        catboost_valid_pred[6],
        catboost_valid_pred[7],

        catboost_valid_pred[9]
    ], axis=0)))

ROC AUC valid:  0.8991240089159537


In [176]:
catboost_valid_pred = deepcopy(predictions)

In [172]:
catboost_models = deepcopy(models)

In [234]:
negatives = np.array_split(df_train[df_train['target']==0], 10)

predictions = []


for i, model in enumerate(catboost_models):
    print(f'Iteration {i+1}')
    train = pd.concat([df_train[df_train['target']==1], negative], axis=0).sample(frac=1)

    X_train = train.drop(['ID_code', 'target'], axis=1)
    y_train = train['target']

    y_pred_train = model.predict_proba(X_train)[:,1]
    y_pred_valid = model.predict_proba(X_valid)[:,1]
    
    predictions.append(y_pred_valid)

    print('ROC AUC train: ', roc_auc_score(y_train, y_pred_train))
    print('ROC AUC valid: ', roc_auc_score(y_valid, y_pred_valid))

Iteration 1
ROC AUC train:  0.9255883050224645
ROC AUC valid:  0.8955294331566181
Iteration 2
ROC AUC train:  0.9528647544464397
ROC AUC valid:  0.8953415800375555
Iteration 3
ROC AUC train:  0.9252235594995553
ROC AUC valid:  0.8949398090149586
Iteration 4
ROC AUC train:  0.9248981725793834
ROC AUC valid:  0.8946182718978759
Iteration 5
ROC AUC train:  0.9257100193839822
ROC AUC valid:  0.8938265730823372
Iteration 6
ROC AUC train:  0.9261229390069481
ROC AUC valid:  0.8949874792242236
Iteration 7
ROC AUC train:  0.925262122979005
ROC AUC valid:  0.8951656116305632
Iteration 8
ROC AUC train:  0.925450420273217
ROC AUC valid:  0.8949261543864888
Iteration 9
ROC AUC train:  0.9251532126952114
ROC AUC valid:  0.8942136385886024
Iteration 10
ROC AUC train:  0.9255984298804152
ROC AUC valid:  0.8956837477426652


# Submission

In [265]:
y_pred_test = [model.predict_proba(X_test)[:,1] for model in models]

In [266]:
y_pred_test

[array([0.51353943, 0.54810137, 0.76105474, ..., 0.03965492, 0.52458607,
        0.3681197 ]),
 array([0.45200251, 0.58584933, 0.72726154, ..., 0.03136979, 0.37520929,
        0.39195908]),
 array([0.48648874, 0.56760553, 0.71321079, ..., 0.04521828, 0.3873024 ,
        0.34407176]),
 array([0.49046804, 0.59266761, 0.75203243, ..., 0.04171874, 0.39158973,
        0.34588926]),
 array([0.48559157, 0.69964847, 0.67615222, ..., 0.03398007, 0.43168128,
        0.29355007]),
 array([0.51502859, 0.6364025 , 0.73461508, ..., 0.02221924, 0.27103338,
        0.27638584]),
 array([0.50268163, 0.70179678, 0.65646144, ..., 0.03283017, 0.43848574,
        0.45370141]),
 array([0.63235574, 0.77388221, 0.69470053, ..., 0.03546403, 0.39659013,
        0.22485201]),
 array([0.61901025, 0.59334106, 0.76609618, ..., 0.02192895, 0.3763465 ,
        0.42818916])]

In [218]:
np.mean([
    y_pred_test[0],
    y_pred_test[1],
    y_pred_test[2],
    y_pred_test[3],

    y_pred_test[5],
    y_pred_test[6],
    y_pred_test[7],

    y_pred_test[9]
], axis=0).shape

(200000,)

In [216]:
def submit(y_pred, filename):
    subm = np.column_stack([df_test['ID_code'], y_pred])
    with open(filename, 'w+') as f:
        f.write('ID_code,target\n')
        for row in subm:
            f.write('{},{}\n'.format(row[0],row[1]))

In [267]:
submit(np.mean(y_pred_test, axis=0), './submissions/catboost_ensemble_sampling_all_set.csv')

# Standard scaler

In [37]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

# MinMax scaler

In [73]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

# LightGBM

In [ ]:
param = {'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1}


cv_split = StratifiedKFold(n_splits=5, random_state=2129)

y_pred_valid_lightgbm = 0*y

lgb_models = []

for train_id, valid_id in cv_split.split(X,y):
    X_train, X_valid = X.iloc[train_id], X.iloc[valid_id]
    y_train, y_valid = y.iloc[train_id], y.iloc[valid_id]
        
    data_train = lgb.Dataset(X_train, label=y_train)
    data_valid = lgb.Dataset(X_valid, label=y_valid)
    
    model = lgb.train(params,
                    data_train,
                    1000000,
                    valid_sets=[data_valid],
                    verbose_eval=1000,
                    early_stopping_rounds=3500)
    
    lgb_models.append(model)

    y_pred_train = model.predict(X_train)
    y_pred_valid = model.predict(X_valid)
    
    y_pred_valid_lightgbm.iloc[valid_id] = y_pred_valid

    print('ROC AUC train: ', roc_auc_score(y_train, y_pred_train))
    print('ROC AUC valid: ', roc_auc_score(y_valid, y_pred_valid))

Training until validation scores don't improve for 3500 rounds.
[1000]	valid_0's auc: 0.855104
[2000]	valid_0's auc: 0.875306
[3000]	valid_0's auc: 0.8839
[4000]	valid_0's auc: 0.888864
[5000]	valid_0's auc: 0.891466
[6000]	valid_0's auc: 0.892394
[7000]	valid_0's auc: 0.892881
[8000]	valid_0's auc: 0.893115
[9000]	valid_0's auc: 0.893244
[10000]	valid_0's auc: 0.89331
[11000]	valid_0's auc: 0.893302
[12000]	valid_0's auc: 0.893312
[13000]	valid_0's auc: 0.893306
[14000]	valid_0's auc: 0.893216
[15000]	valid_0's auc: 0.893159


In [67]:
data_train = lgb.Dataset(X_train, label=y_train)
data_valid = lgb.Dataset(X_valid, label=y_valid)

param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1,
    'random_state': 2129,
}

clf = lgb.train(params,
                data_train,
                20000,
                valid_sets=[data_valid],
                verbose_eval=100,
                early_stopping_rounds=5000)

Training until validation scores don't improve for 5000 rounds.
[100]	valid_0's auc: 0.756398
[200]	valid_0's auc: 0.79136
[300]	valid_0's auc: 0.809593
[400]	valid_0's auc: 0.821838
[500]	valid_0's auc: 0.831376
[600]	valid_0's auc: 0.838722
[700]	valid_0's auc: 0.844718
[800]	valid_0's auc: 0.850024
[900]	valid_0's auc: 0.854257
[1000]	valid_0's auc: 0.858013
[1100]	valid_0's auc: 0.861273
[1200]	valid_0's auc: 0.864301
[1300]	valid_0's auc: 0.866846
[1400]	valid_0's auc: 0.869118
[1500]	valid_0's auc: 0.871131
[1600]	valid_0's auc: 0.873031
[1700]	valid_0's auc: 0.874626
[1800]	valid_0's auc: 0.876188
[1900]	valid_0's auc: 0.877428
[2000]	valid_0's auc: 0.878591
[2100]	valid_0's auc: 0.879738
[2200]	valid_0's auc: 0.880765
[2300]	valid_0's auc: 0.881741
[2400]	valid_0's auc: 0.882605
[2500]	valid_0's auc: 0.883424
[2600]	valid_0's auc: 0.884203
[2700]	valid_0's auc: 0.884926
[2800]	valid_0's auc: 0.885642
[2900]	valid_0's auc: 0.886305
[3000]	valid_0's auc: 0.886877
[3100]	valid_0's

In [39]:
y_pred_train = clf.predict(X_train)
y_pred_valid = clf.predict(X_valid)
print('ROC AUC train: ', roc_auc_score(y_train, y_pred_train))
print('ROC AUC valid: ', roc_auc_score(y_valid, y_pred_valid))

ROC AUC train:  0.9742343115775871
ROC AUC valid:  0.8953884759083959


In [82]:
y_pred_train = catboost.predict_proba(X_train)[:,1]
y_pred_valid = catboost.predict_proba(X_valid)[:,1]
print('ROC AUC train: ', roc_auc_score(y_train, y_pred_train))
print('ROC AUC valid: ', roc_auc_score(y_valid, y_pred_valid))

ROC AUC train:  0.946276533090301
ROC AUC valid:  0.8902662410570826


In [96]:
y_pred_train = catboost.predict_proba(X_train)[:,1]
y_pred_valid = catboost.predict_proba(X_valid)[:,1]
print('ROC AUC train: ', roc_auc_score(y_train, y_pred_train))
print('ROC AUC valid: ', roc_auc_score(y_valid, y_pred_valid))

ROC AUC train:  0.9207569001708658
ROC AUC valid:  0.8937263653937096
